In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.display import clear_output
import time
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate
from tqdm import tqdm
sym.init_session(use_latex=True)

# Parcial 3